# Extracting and Visualizing Stock Data (Tesla & GameStop)
This notebook follows the IBM Skills Network lab format and contains all 7 questions:
1. Extract Tesla stock data using `yfinance`
2. Extract Tesla revenue data using web scraping (Macrotrends)
3. Extract GameStop stock data using `yfinance`
4. Extract GameStop revenue data using web scraping (Macrotrends)
5. Tesla stock vs revenue dashboard
6. GameStop stock vs revenue dashboard
7. Save & share notebook

Run each code cell in order. If you open this notebook in an environment without `yfinance` or internet access, some cells may fail — they are written to match the lab assignment exactly.

## Question 1 — Extracting Tesla Stock Data Using yfinance
Create a ticker object and download the full history for Tesla (TSLA).

In [ ]:
import yfinance as yf
import pandas as pd

# Create a ticker object for Tesla
tesla = yf.Ticker("TSLA")

# Extract Tesla stock data for the maximum available period
tesla_data = tesla.history(period="max")

# Reset the index to make Date a column instead of an index
tesla_data.reset_index(inplace=True)

# Display the first five rows
tesla_data.head()


## Question 2 — Extracting Tesla Revenue Data Using Web Scraping
Use Macrotrends Tesla revenue page to extract the quarterly revenue table.

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html.parser")

# Find the revenue table
tables = soup.find_all("table")
tesla_revenue = None
for table in tables:
    if "Tesla Quarterly Revenue" in str(table) or "Tesla Quarterly Revenue" in table.text:
        tesla_revenue = table

if tesla_revenue is None:
    # Try another heuristic: find table with two columns 'Date' and 'Revenue'
    for table in tables:
        text = table.text.lower()
        if 'revenue' in text and 'date' in text:
            tesla_revenue = table

if tesla_revenue is None:
    raise ValueError('Could not find Tesla revenue table on Macrotrends page')

tesla_revenue_data = pd.read_html(str(tesla_revenue))[0]
tesla_revenue_data.columns = ["Date", "Revenue"]

# Clean Revenue: remove $ and commas
tesla_revenue_data["Revenue"] = tesla_revenue_data["Revenue"].replace({'\$':'', ',':''}, regex=True)
tesla_revenue_data.dropna(inplace=True)
tesla_revenue_data = tesla_revenue_data[tesla_revenue_data["Revenue"] != ""]

tesla_revenue_data.head()


## Question 3 — Extracting GameStop Stock Data Using yfinance
Create a ticker object and download the full history for GameStop (GME).

In [ ]:
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)
gme_data.head()


## Question 4 — Extracting GameStop Revenue Data Using Web Scraping
Use Macrotrends GameStop revenue page to extract the quarterly revenue table.

In [ ]:
url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html.parser")

tables = soup.find_all("table")
gme_revenue = None
for table in tables:
    if "GameStop Quarterly Revenue" in str(table) or "GameStop Quarterly Revenue" in table.text:
        gme_revenue = table

if gme_revenue is None:
    for table in tables:
        text = table.text.lower()
        if 'revenue' in text and 'date' in text:
            gme_revenue = table

if gme_revenue is None:
    raise ValueError('Could not find GameStop revenue table on Macrotrends page')

gme_revenue_data = pd.read_html(str(gme_revenue))[0]
gme_revenue_data.columns = ["Date", "Revenue"]
gme_revenue_data["Revenue"] = gme_revenue_data["Revenue"].replace({'\$':'', ',':''}, regex=True)
gme_revenue_data.dropna(inplace=True)
gme_revenue_data = gme_revenue_data[gme_revenue_data["Revenue"] != ""]

gme_revenue_data.head()


## Question 5 — Tesla Stock and Revenue Dashboard
Plot Tesla Close price and Revenue on a dual-axis chart.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

# Convert revenue Date to datetime if needed
try:
    tesla_revenue_data['Date'] = pd.to_datetime(tesla_revenue_data['Date'])
except Exception:
    pass

fig, ax1 = plt.subplots(figsize=(12,6))
ax1.plot(tesla_data['Date'], tesla_data['Close'], label='Tesla Close')
ax1.set_xlabel('Date')
ax1.set_ylabel('Stock Price (USD)')

ax2 = ax1.twinx()
ax2.plot(tesla_revenue_data['Date'], tesla_revenue_data['Revenue'].astype(float), color='orange', label='Revenue')
ax2.set_ylabel('Revenue (USD)')

plt.title('Tesla Stock Price vs Revenue')
ax1.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
fig.autofmt_xdate()
plt.show()


## Question 6 — GameStop Stock and Revenue Dashboard
Plot GameStop Close price and Revenue on a dual-axis chart.

In [ ]:
try:
    gme_revenue_data['Date'] = pd.to_datetime(gme_revenue_data['Date'])
except Exception:
    pass

fig, ax1 = plt.subplots(figsize=(12,6))
ax1.plot(gme_data['Date'], gme_data['Close'], label='GME Close')
ax1.set_xlabel('Date')
ax1.set_ylabel('Stock Price (USD)')

ax2 = ax1.twinx()
ax2.plot(gme_revenue_data['Date'], gme_revenue_data['Revenue'].astype(float), color='orange', label='Revenue')
ax2.set_ylabel('Revenue (USD)')

plt.title('GameStop Stock Price vs Revenue')
ax1.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
fig.autofmt_xdate()
plt.show()


## Question 7 — Save & Share
Save this notebook as `Tesla_GameStop_Final_Project.ipynb`. Take screenshots of code+output and the two dashboards and upload them to the assignment submission.

In [ ]:
print('Notebook ready. Run all cells to produce outputs and screenshots for submission.')
